## 1 - Collect dataset
Connect the left and right pins of the potentiometer to the ground and 3.3V pins in Bela and the middle pin to the analog input A0. Run the `dataset-capture` project on Bela

In [ ]:
from miniPyBela import Streamer

In [ ]:
streamer=Streamer(ip="bela.local")
streamer.start_streaming(saving_enabled=True)

In [ ]:
streamer.stop_streaming()

## 2 - Train model

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from tqdm import tqdm 
import pprint as pp
from torch.utils.data import Dataset, DataLoader
from pyBela import Streamer
from torch.utils.mobile_optimizer import optimize_for_mobile


In [ ]:
streamer = Streamer()
raw = streamer.load_data_from_file("pot_var_stream__3.txt")
data = [data for buffer in raw for data in buffer["data"]]

class PotentiometerDataset(Dataset):
    def __init__(self, data, seq_len=32):
        super().__init__()
        
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        # make len divisible by seq_len
        data = data[:len(data) - (len(data) % seq_len)]
        sequences = [data[i:i+seq_len] for i in range(0, len(data), seq_len)]

        self.inputs = torch.tensor(sequences[:-1]).float().to(self.device)
        self.outputs = torch.tensor(sequences[1:]).float().to(self.device)
        
    def __len__(self):
        return len(self.inputs)
    
    def __getitem__(self, i):
        return self.inputs[i].unsqueeze(dim=1), self.outputs[i].unsqueeze(dim=1)
    
dataset = PotentiometerDataset(data, seq_len=32)
_in, out = dataset.__getitem__(0)

print(_in.shape, out.shape)


# Split dataset
train_count = int(0.9 * dataset.__len__())
test_count = dataset.__len__() - train_count
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_count, test_count)
)

batch_size = 64
# Dataloaders
train_loader = DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    test_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True, nonlinearity='relu')
        self.fc = nn.Linear(hidden_size, output_size)
        
    def forward(self, x):
        # Initialize hidden state with zeros
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device)
        
        # Forward propagate the RNN
        out, _ = self.rnn(x, h0)
        
        # Apply the linear layer to get the final output
        out = self.fc(out)
        return out
    
#model = RNN(input_size=1, hidden_size=64, out_size=1).to(device='cuda')
#model = nn.RNN(input_size=1, hidden_size=12, num_layers=1).to(device='cuda')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SimpleRNN(input_size=1, hidden_size=64, output_size=1).to(device=device)
optimizer = torch.optim.SGD(
    model.parameters(), lr=0.001)
criterion = torch.nn.MSELoss(reduction='mean')

In [ ]:
epochs = 50

print("Running on device: {}".format(device))
for epoch in range(1, epochs+1):

    print("█▓░ Epoch: {} ░▓█".format(epoch))

    # training loop
    train_it_losses = np.array([])
    model.train()

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # (batch_size, seq_len, input_size)
        data = data.to(device=device, non_blocking=True)
        # (batch_size, seq_len, input_size)
        targets = targets.to(device=device, non_blocking=True)
        
        

        optimizer.zero_grad(set_to_none=True)  # lower memory footprint
        out = model(data)
        train_loss = torch.sqrt(criterion(out, targets))
        train_it_losses = np.append(train_it_losses, train_loss.item())
        train_loss.backward()
        optimizer.step()

    # test loop
    test_it_losses = []

    for batch_idx, (data, targets) in enumerate(tqdm(test_loader)):
        # (batch_size, seq_length, input_size)
        data = data.to(device=device, non_blocking=True)
        # (batch_size, seq_length, out_size)
        targets = targets.to(device=device, non_blocking=True)
        model.eval()
        with torch.no_grad():
            out = model(data)  # using predict method to avoid backprop
        test_loss = torch.sqrt(criterion(out, targets))
        test_it_losses = np.append(
            test_it_losses, test_loss.item())

    losses = {"train_loss": train_it_losses.mean().round(
        8), "test_loss": test_it_losses.mean().round(8)}
    pp.pprint(losses, sort_dicts=False)


In [ ]:
#plot predictions on test data

import matplotlib.pyplot as plt
input, target = test_dataset.__getitem__(5)

output = model(input.unsqueeze(0))

print(input.unsqueeze(0).shape, output.shape)
print(input.shape, target.shape, output.shape)

# Plot the results
plt.figure(figsize=(10, 6))
plt.plot(target.view(-1).detach().cpu(), label='True')
plt.plot(output.view(-1).detach().cpu(), label='Predictions')
plt.xlabel('Time')
plt.ylabel('Value')
plt.legend()
plt.ylim(0, 3)
plt.show()


In [ ]:
model.to(device='cpu')
model.eval()
script = torch.jit.script(model)
script.save("bela-code/model.jit")


In [ ]:
torch.jit.load("model.jit") # check model is properly saved

## 3 - deploy and run